In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import requests

In [2]:
#Sempre que o chrome drvier for aberto o mesmo irá atualizar atomaticamente.
#driver = webdriver.Chrome(ChromeDriverManager().install())

In [1]:
!pip3 install webdriver-manager==3.3.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached webdriver_manager-3.3.0-py2.py3-none-any.whl (16 kB)
  Using cached crayons-0.4.0-py2.py3-none-any.whl (4.6 kB)
  Using cached configparser-6.0.0-py3-none-any.whl (19 kB)


In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Mar  3 00:18:28 2021

@author: pauloeduardosampaio
"""

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import pandas as pd
from tqdm import tqdm
from datetime import datetime

driver = webdriver.Chrome(ChromeDriverManager().install())

urls_dict = {"Brooklin": "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-sul/brooklin/apartamento_residencial/",
             "Butanta": "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-oeste/butanta/apartamento_residencial/",
             "Republica": "https://www.vivareal.com.br/venda/sp/sao-paulo/centro/republica/apartamento_residencial/",
             "Bras": "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-leste/bras/apartamento_residencial/",
             "Fregesia do O": "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-norte/freguesia-do-o/apartamento_residencial/",
             "Higienopolis": "https://www.vivareal.com.br/venda/sp/sao-paulo/centro/higienopolis/apartamento_residencial/",
             "Jardins": "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-oeste/jardins/apartamento_residencial/",
             "Lapa": "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-oeste/lapa/apartamento_residencial/",
             "Moema": "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-sul/moema/apartamento_residencial/", 
             "Mooca": "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-leste/mooca/apartamento_residencial/",
             "Morumbi": "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-sul/morumbi/apartamento_residencial/",
             "Pinheiros": "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-oeste/pinheiros/apartamento_residencial/",
             "Santana": "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-norte/santana/apartamento_residencial/",
             "Saude": "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-sul/saude/apartamento_residencial/",
             "Vila Mariana": "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-sul/vila-mariana/apartamento_residencial/",
             "Vila Matilde": "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-leste/vila-matilde/apartamento_residencial/",
             "Campo limpo": "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-sul/campo-limpo/apartamento_residencial/",
             "Tucuruvi": "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-norte/tucuruvi/apartamento_residencial/"}

errors = []
results = []

for bairro, url in urls_dict.items():
    current_url = url
    driver.get(url)
    sleep(2)
    actions = ActionChains(driver)

    try:
        driver.find_element_by_class_name("cookie-notifier__cta").click()
    except:
            print("No cookies!")
    
    for i in tqdm(range(30), desc=bairro):
        
        sleep(5)
        main_div = driver.find_element_by_class_name("results-main__panel")
        properties = main_div.find_elements_by_class_name("js-property-card")
        paginator = driver.find_element_by_class_name("js-results-pagination")
        next_page = paginator.find_element_by_xpath('//*[@id="js-site-main"]/div[2]/div[1]/section/div[2]/div[2]/div/ul/li[9]/button')
        
        for i,apartment in enumerate(properties):
            url = apartment.find_element_by_class_name("js-card-title").get_attribute("href")
            apto_id = url.split("id-")[-1][:-1]
            header = apartment.find_element_by_class_name("property-card__title").text
            address = apartment.find_element_by_class_name("property-card__address").text
            area = apartment.find_element_by_class_name("js-property-card-detail-area").text
            rooms = apartment.find_element_by_class_name("js-property-detail-rooms").text
            bathrooms = apartment.find_element_by_class_name("js-property-detail-bathroom").text
            garages = apartment.find_element_by_class_name("js-property-detail-garages").text
            try:
                amenities = apartment.find_element_by_class_name("property-card__amenities").text
            except:
                amenities = None
                errors.append(url)
            price = apartment.find_element_by_class_name("js-property-card-prices").text
            try:
                condo = apartment.find_element_by_class_name("js-condo-price").text
            except:
                condo = None
                errors.append(url)
            crawler = bairro
            crawled_at = datetime.now().strftime("%Y-%m-%d %H:%M")
            results.append({"id": apto_id,
                            "url": url,
                            "header": header,
                            "address": address,
                            "area": area,
                            "rooms": rooms,
                            "bathrooms": bathrooms,
                            "garages": garages,
                            "amenities": amenities,
                            "price": price,
                            "condo": condo,
                            "crawler": crawler,
                            "crawled_at": crawled_at})
        next_page.click()
pd.DataFrame(results).to_csv("full_results.csv", index=False)
driver.close()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 114.0.5735
[WDM] - Get LATEST driver version for 114.0.5735


[WDM] - Driver [/home/thiago-vale/.wdm/drivers/chromedriver/linux64/114.0.5735.90/chromedriver] found in cache
Brooklin: 100%|██████████| 30/30 [06:56<00:00, 13.88s/it]


No cookies!


Butanta:  10%|█         | 3/30 [00:52<07:48, 17.36s/it]


KeyboardInterrupt: 

In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import pandas as pd
from tqdm import tqdm
from datetime import datetime

def extract_apartment_data(bairro, url):
    driver = webdriver.Chrome(ChromeDriverManager().install()) # Inicializa o driver do Chrome
    errors = []  # Lista para armazenar URLs que causaram erros durante a extração
    results = []  # Lista para armazenar os resultados extraídos dos apartamentos

    current_url = url
    driver.get(url) # Abre a URL no navegador
    sleep(2)
    actions = ActionChains(driver) # Cria uma instância de ActionChains para ações de mouse

    try:
        driver.find_element_by_class_name("cookie-notifier__cta").click() # Fecha a mensagem de notificação de cookies, se existir
    except:
        print("No cookies!") # Exibe uma mensagem se não houver mensagem de notificação de cookies

    for i in tqdm(range(30), desc=bairro):
        sleep(5)
        main_div = driver.find_element_by_class_name("results-main__panel") # Encontra a seção principal de resultados
        properties = main_div.find_elements_by_class_name("js-property-card") # Encontra os elementos que representam os apartamentos na página
        paginator = driver.find_element_by_class_name("js-results-pagination") # Encontra o elemento que contém o paginador
        next_page = paginator.find_element_by_xpath('//*[@id="js-site-main"]/div[2]/div[1]/section/div[2]/div[2]/div/ul/li[9]/button') # Encontra o botão "próxima página"

        for i, apartment in enumerate(properties): # Loop para extrair informações de cada apartamento na página
            url = apartment.find_element_by_class_name("js-card-title").get_attribute("href") # Extrai a URL do apartamento
            apto_id = url.split("id-")[-1][:-1]  # Extrai o ID do apartamento a partir da URL
            header = apartment.find_element_by_class_name("property-card__title").text # Extrai o título do apartamento
            address = apartment.find_element_by_class_name("property-card__address").text  # Extrai o endereço do apartamento
            area = apartment.find_element_by_class_name("js-property-card-detail-area").text # Extrai a área do apartamento
            rooms = apartment.find_element_by_class_name("js-property-detail-rooms").text # Extrai o número de quartos do apartamento
            bathrooms = apartment.find_element_by_class_name("js-property-detail-bathroom").text  #Extrai o número de banheiros do apartamento
            garages = apartment.find_element_by_class_name("js-property-detail-garages").text # Extrai o número de vagas de garagem do apartamento
            try:
                amenities = apartment.find_element_by_class_name("property-card__amenities").text # Extrai as comodidades do apartamento, se estiverem disponíveis
            except:
                amenities = None
                errors.append(url)  # Adiciona a URL à lista de erros se ocorrer um erro ao extrair as comodidades
            price = apartment.find_element_by_class_name("js-property-card-prices").text # Extrai o preço do apartamento
            try:
                condo = apartment.find_element_by_class_name("js-condo-price").text # Extrai a taxa do condomínio, se estiver disponível
            except:
                condo = None
                errors.append(url) # Adiciona a URL à lista de erros se ocorrer um erro ao extrair a taxa do condomínio

            crawler = bairro # Define o bairro como o nome do crawler
            crawled_at = datetime.now().strftime("%Y-%m-%d %H:%M")  # Obtém a data e hora atual formatada

            results.append({"id": apto_id,
                            "url": url,
                            "header": header,
                            "address": address,
                            "area": area,
                            "rooms": rooms,
                            "bathrooms": bathrooms,
                            "garages": garages,
                            "amenities": amenities,
                            "price": price,
                            "condo": condo,
                            "crawler": crawler,
                            "crawled_at": crawled_at}) # Adiciona um dicionário com as informações do apartamento à lista de resultados
        next_page.click()  # Clica no botão "próxima página" para avançar para a próxima página de resultados
    driver.close() # Fecha o navegador
    return results



bairro = "Brooklin"
url = "https://www.vivareal.com.br/venda/sp/sao-paulo/zona-sul/brooklin/apartamento_residencial/"
results = extract_apartment_data(bairro, url)
pd.DataFrame(results).to_csv("full_results.csv", index=False) # Converte a lista de resultados em um DataFrame e salva em um arquivo CSV

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 114.0.5735
[WDM] - Get LATEST driver version for 114.0.5735


[WDM] - Driver [/home/thiago-vale/.wdm/drivers/chromedriver/linux64/114.0.5735.90/chromedriver] found in cache
Brooklin: 100%|██████████| 30/30 [07:19<00:00, 14.66s/it]
